# Connecting to MongoDB

This notebook shows how one can connect to a MongoDB instance.

In [1]:
# Import system package.
import sys, os
import urllib.parse
import datetime

In [1]:
# Import pymongo
try:
    from pymongo import MongoClient
except:
    !{sys.executable} -m pip install pymongo
    from pymongo import MongoClient

# Import dotenv
try:
    from dotenv import dotenv_values
except:
    !{sys.executable} -m pip install "python-dotenv"
    from dotenv import dotenv_values
    
# Import bson
try:
    from bson.objectid import ObjectId
except:
    !{sys.executable} -m pip install bson
    from bson.objectid import ObjectId
    
# Import xlrd
try:
    import xlrd
except:
    !{sys.executable} -m pip install xlrd
    
# Import openpyxl
try:
    import openpyxl
except:
    !{sys.executable} -m pip install openpyxl

In [12]:
# Load the environment variables.
ENV_PATH = "./../.env"
config = { 
    **dotenv_values(ENV_PATH + ".config"),
    **dotenv_values(ENV_PATH + ".secrets"),
    **os.environ,
}

You might notice I am really loading two files here. You should have two environment files:

`.env.config` that contains the following:
```.env
LOOKUP_TABLE = "edu.iste.disabilityresourcesearchengine.xlsx"
LOOKUP_SHEETS = "disability_category,service_category,states,zipcodes"

OH_PROVIDERS = "org.handson.ohio.serviceproviders.json"
NY_PROVIDERS = "gov.nys.serviceproviders.xlsx"
CA_PROVIDERS = "gov.ca.serviceproviders.xlsx"
```

and `.env.secrets` that contains your authentication information:

```dotenv
DB_HOST = "<ip address or hostname>"
DB_USER = "<username>"
DB_PASS = "<password>"
DB_PORT = <port>
```

In [4]:
# Get the MongoDB connection string.
def get_connection_string(options):
    hostname = urllib.parse.quote_plus(options['DB_HOST'])
    username = urllib.parse.quote_plus(options['DB_USER'])
    password = urllib.parse.quote_plus(options['DB_PASS'])
    port = int(options['DB_PORT'])
    return "mongodb://%s:%s@%s:%d/?authSource=admin" % (username, password, hostname, port)

# If client doesn't exist, attempt to connect.
try:
    client
except:
    # Prepare MongoDB connection string.
    uri = get_connection_string(config)
    # Prepare MongoDB Client.
    client = MongoClient(uri)

In [5]:
# Display MongoDB client details.
client

MongoClient(host=['mongodb.thelionsredmane.stream:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin')

In [6]:
# Databases and collections are lazily initialized: just refer to the one you want.
test_db = client.test
test_coll = test_db.test_coll

# Documents do not enforce structure.
test_entry = {
    "author": "Ian Effendi",
    "text": "My first post!",
    "tags": ["this", "is", "a", "entry"],
    "date": datetime.datetime.utcnow()
}

# Create collections (and their related databases) in the server, once you perform a write operation.
response = test_coll.insert_one(test_entry)
response

In [7]:
# Use the inserted_id property to get an ObjectId.
response.inserted_id

ObjectId('610998c6bb306080a74df7cd')

In [8]:
# See a list of collections on the server.
test_db.list_collection_names()

['test_coll']

In [9]:
# Use `find_one()` to query the database.
test_coll.find_one()

{'_id': ObjectId('610990132bccf24343c9f52a'),
 'author': 'Ian Effendi',
 'text': 'My first post!',
 'tags': ['this', 'is', 'a', 'entry'],
 'date': datetime.datetime(2021, 8, 3, 18, 50, 59, 93000)}